In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [3]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [4]:
# !kaggle datasets download -d dm4006/sent-emb-result

In [5]:
# !unzip \*.zip  && rm *.zip

In [6]:
!ls

kaggle.json


In [7]:
import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [8]:
train = pd.read_csv("/content/drive/MyDrive/data/csv/train.csv")

In [ ]:
# /content/drive/MyDrive

In [9]:
train.shape

(87599, 4)

In [10]:
with open("/content/drive/MyDrive/data/weights/dict_embeddings1.pickle", "rb") as f:
    d1 = pickle.load(f)

In [11]:
with open("/content/drive/MyDrive/data/weights/dict_embeddings2.pickle", "rb") as f:
    d2 = pickle.load(f)

In [12]:
len(d1), len(d2), type(d1)

(90743, 90742, dict)

In [13]:
dict_emb = dict(d1)
dict_emb.update(d2)

In [14]:
len(dict_emb), type(dict_emb)

(181485, dict)

In [15]:
del d1, d2

In [16]:
train.head(3)

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building


In [17]:
train.dropna(inplace=True)

In [18]:
train.shape

(87598, 4)

In [19]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train

In [22]:
train_process_data = process_data(train)

step 1
step 2
step 3
step 4


In [21]:
# train_process_data.to_csv("/content/drive/MyDrive/data/temp/train_process_data.csv", index = None)

In [6]:
# train_process_data = pd.read_csv("/content/drive/MyDrive/data/temp/train_process_data.csv")

In [23]:
del train

In [24]:
train_process_data.shape

(87598, 8)

In [54]:
print(train_process_data.iloc[87597])

answer_start                                                    0
context         Kathmandu Metropolitan City (KMC), in order to...
question                            What is KMC an initialism of?
text                                  Kathmandu Metropolitan City
sentences       [Kathmandu Metropolitan City (KMC), in order t...
target                                                          0
sent_emb        [[0.061106615, 0.11834025, 0.080840126, -0.010...
quest_emb       [[0.07114331, 0.10495965, -0.0062274314, -0.02...
Name: 87598, dtype: object


In [55]:
train_process_data.head()

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb
0,515,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.11010082, 0.114229426, 0.115608975, 0.0548..."
1,188,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.10951652, 0.110306256, 0.052100066, 0.0305..."
2,279,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.0119564785, 0.14930709, 0.026600497, 0.052..."
3,381,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.07114331, 0.05411832, -0.013959836, 0.0531..."
4,92,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.16133595, 0.15039583, 0.09225755, 0.040458..."


In [26]:
train_process_data = train_process_data[['answer_start','context','question','text','sentences','target','sent_emb','quest_emb']]
train_process_data.head()

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb
0,515,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.11010082, 0.114229426, 0.115608975, 0.0548..."
1,188,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.10951652, 0.110306256, 0.052100066, 0.0305..."
2,279,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.0119564785, 0.14930709, 0.026600497, 0.052..."
3,381,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.07114331, 0.05411832, -0.013959836, 0.0531..."
4,92,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.16133595, 0.15039583, 0.09225755, 0.040458..."


In [61]:
# splitting dataframe by row index 
df_1 = train_process_data.iloc[0:15000]
df_2 = train_process_data.iloc[15000:30000]
df_3 = train_process_data.iloc[30000:45000]
df_4 = train_process_data.iloc[45000:60000]
df_5 = train_process_data.iloc[60000:75000]
df_6 = train_process_data.iloc[75000:87597]
# df_2 = train_process_data.iloc[1001:,:] 
# print("Shape of new dataframes - {} , {}".format(df_1.shape, df_2.shape)) 

In [62]:
df_1.shape, df_6.shape , df_4.shape

((15000, 8), (12597, 8), (15000, 8))

In [63]:
df_1.tail(5)

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb
14995,79,Several universities located outside Boston ha...,What Univesity located outside of Boston still...,Harvard University,[Several universities located outside Boston h...,1,"[[0.17808884, 0.09535843, 0.04898285, -0.00944...","[[0.14361657, 0.11795563, 0.029925916, -0.0233..."
14996,79,Several universities located outside Boston ha...,What is the nations oldest college?,Harvard University,[Several universities located outside Boston h...,1,"[[0.17808884, 0.09535843, 0.04898285, -0.00944...","[[0.11431218, 0.05411832, -0.013959836, 0.0854..."
14997,389,Several universities located outside Boston ha...,Where does Harvard plan to expand to?,Allston,[Several universities located outside Boston h...,3,"[[0.17808884, 0.09535843, 0.04898285, -0.00944...","[[0.039967835, 0.043929018, -0.011820921, -0.0..."
14998,581,Several universities located outside Boston ha...,What was MIT originally known as?,Boston Tech,[Several universities located outside Boston h...,4,"[[0.17808884, 0.09535843, 0.04898285, -0.00944...","[[0.020957481, 0.14156649, 0.028817743, -0.034..."
14999,190,Several universities located outside Boston ha...,"In 1916, MIT moved across the river to what city?",Cambridge,[Several universities located outside Boston h...,4,"[[0.17808884, 0.09535843, 0.04898285, -0.00944...","[[0.0786925, 0.14032005, 0.033017013, 0.042616..."


In [64]:
df_2.head(5)

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb
15000,70,"Like many major American cities, Boston has se...","Since the early 1990's, Boston has had less of...",violent crime,"[Like many major American cities, Boston has s...",0,"[[0.1391201, 0.11856326, 0.11958357, 0.0994465...","[[0.0639045, 0.11707643, 0.0431389, 0.05189094..."
15001,281,"Like many major American cities, Boston has se...",Boston police and local groups make an effort ...,joining gangs,"[Like many major American cities, Boston has s...",1,"[[0.1391201, 0.11856326, 0.11958357, 0.0994465...","[[0.038353834, 0.0524202, 0.06988523, 0.048463..."
15002,458,"Like many major American cities, Boston has se...",The Boston Miracle is the name for the reducti...,Murders,"[Like many major American cities, Boston has s...",3,"[[0.1391201, 0.11856326, 0.11958357, 0.0994465...","[[0.048519164, 0.11784551, 0.09717072, 0.01545..."
15003,525,"Like many major American cities, Boston has se...",What was the murder rate in Boston in 1990?,"26.5 per 100,000","[Like many major American cities, Boston has s...",3,"[[0.1391201, 0.11856326, 0.11958357, 0.0994465...","[[0.084122546, 0.07777708, 0.074291766, 0.1470..."
15004,20,"In 2008, there were 62 reported homicides. Thr...",How many reported murders were there in 2008?,62,"[In 2008, there were 62 reported homicides., T...",0,"[[0.037050683, 0.0970713, 0.12121038, 0.071857...","[[0.08400851, 0.07712097, 0.09478019, 0.089347..."


In [65]:
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li

In [66]:
def pred_idx(distances):
    return np.argmin(distances)


In [68]:
def predictions(train):
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diff"]
    # print("cosine start")
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    return train


In [71]:
predicted1 = predictions(df_1)
predicted2 = predictions(df_2)
predicted3 = predictions(df_3)
predicted4 = predictions(df_4)
predicted5 = predictions(df_5)
predicted6 = predictions(df_6)

In [72]:
del df_1, df_2, df_3, df_4, df_5, df_6

In [73]:
predicted1.to_csv("/content/drive/MyDrive/data/temp/predicted1.csv", index = None)
predicted2.to_csv("/content/drive/MyDrive/data/temp/predicted2.csv", index = None)
predicted3.to_csv("/content/drive/MyDrive/data/temp/predicted3.csv", index = None)
predicted4.to_csv("/content/drive/MyDrive/data/temp/predicted4.csv", index = None)
predicted5.to_csv("/content/drive/MyDrive/data/temp/predicted5.csv", index = None)
predicted6.to_csv("/content/drive/MyDrive/data/temp/predicted6.csv", index = None)

In [44]:
# predicted = pd.read_csv("/content/drive/MyDrive/data/temp/predicted.csv")

In [74]:
predicted1.shape,predicted2.shape ,predicted3.shape ,predicted5.shape ,predicted6.shape

((15000, 12), (15000, 12), (15000, 12), (15000, 12), (12597, 12))

In [125]:
predicted1.head(10)

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,515,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.11010082, 0.114229426, 0.115608975, 0.0548...","[0.42473626136779785, 0.364050030708313, 0.347...","[14.56386, 15.262213, 17.398178, 14.272491, 13...",5,5
1,188,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.10951652, 0.110306256, 0.052100066, 0.0305...","[0.45407456159591675, 0.32262009382247925, 0.3...","[12.889507, 12.285219, 16.843704, 8.361172, 11...",3,3
2,279,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.0119564785, 0.14930709, 0.026600497, 0.052...","[0.3958578109741211, 0.29170823097229004, 0.30...","[11.857298, 11.39232, 15.061657, 7.184714, 8.4...",3,3
3,381,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.07114331, 0.05411832, -0.013959836, 0.0531...","[0.4900696873664856, 0.4060605764389038, 0.456...","[13.317538, 15.017247, 20.81268, 10.511388, 10...",3,3
4,92,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.055199966, 0.050131403, 0.04787037, 0.0162...","[[0.16133595, 0.15039583, 0.09225755, 0.040458...","[0.4777514934539795, 0.289111852645874, 0.3414...","[15.0888195, 11.612733, 16.684145, 9.71824, 12...",3,3
5,248,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,September 1876,"[As at most other universities, Notre Dame's s...",2,"[[0.097203255, 0.09345726, 0.054660242, 0.0484...","[[0.016918724, 0.12084099, 0.013292061, 0.0587...","[0.2747580409049988, 0.3731493353843689, 0.280...","[11.473504, 16.305737, 14.419686, 11.785967, 1...",5,4
6,441,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,twice,"[As at most other universities, Notre Dame's s...",3,"[[0.097203255, 0.09345726, 0.054660242, 0.0484...","[[0.07944554, 0.110715754, 0.11615734, 0.04506...","[0.29136359691619873, 0.44691193103790283, 0.3...","[12.094654, 19.268333, 17.051125, 12.115432, 1...",5,4
7,598,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,The Observer,"[As at most other universities, Notre Dame's s...",9,"[[0.097203255, 0.09345726, 0.054660242, 0.0484...","[[0.07114331, 0.05411832, 0.026413988, 0.08664...","[0.24287956953048706, 0.38149863481521606, 0.3...","[10.405749, 17.056553, 16.048374, 12.674274, 1...",5,0
8,126,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,three,"[As at most other universities, Notre Dame's s...",9,"[[0.097203255, 0.09345726, 0.054660242, 0.0484...","[[0.06699269, 0.050647143, 0.11810354, 0.06676...","[0.18055576086044312, 0.3603665828704834, 0.34...","[7.814688, 16.114157, 17.537537, 12.886265, 13...",0,0
9,908,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,1987,"[As at most other universities, Notre Dame's s...",7,"[[0.097203255, 0.09345726, 0.054660242, 0.0484...","[[0.042654395, 0.13311043, 0.112292886, 0.0977...","[0.2252199649810791, 0.36460912227630615, 0.26...","[9.867832, 16.70340

In [76]:
predicted3.head()

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
30000,302,The sensitivity of Earth-based infrared telesc...,To somewhat avoid the water vapor in the atmos...,at a high altitude,[The sensitivity of Earth-based infrared teles...,1,"[[0.030395534, 0.128848, 0.13439284, 0.0887381...","[[0.04233698, 0.088496, 0.123746514, -0.020588...","[0.20056015253067017, 0.23409879207611084, 0.2...","[11.106165, 10.94219, 10.919176]",0,2
30001,364,The sensitivity of Earth-based infrared telesc...,"Along with aircraft, what object can be used t...",balloon,[The sensitivity of Earth-based infrared teles...,1,"[[0.030395534, 0.128848, 0.13439284, 0.0887381...","[[0.04097561, 0.010560125, 0.10392286, 0.03262...","[0.27886420488357544, 0.1981877088546753, 0.25...","[14.767569, 9.263105, 10.590797]",1,1
30002,388,The sensitivity of Earth-based infrared telesc...,What sorts of telescopes completely avoid wate...,Space telescopes,[The sensitivity of Earth-based infrared teles...,2,"[[0.030395534, 0.128848, 0.13439284, 0.0887381...","[[0.07507865, 0.12538198, 0.11734992, 0.029870...","[0.1989229917526245, 0.2849538326263428, 0.256...","[11.021465, 13.331864, 11.242429]",0,0
30003,0,Near-infrared is the region closest in wavelen...,What is the nearest wavelength to the radiatio...,Near-infrared,[Near-infrared is the region closest in wavele...,0,"[[0.13504988, 0.06811689, 0.17221878, 0.015588...","[[0.07114332, 0.06124491, 0.10257782, 0.015134...","[0.22483479976654053, 0.33156734704971313, 0.4...","[9.655646, 19.930227, 14.265144]",0,0
30004,254,Near-infrared is the region closest in wavelen...,Along with the emission peaks and vs. bands me...,water absorption,[Near-infrared is the region closest in wavele...,1,"[[0.13504988, 0.06811689, 0.17221878, 0.015588...","[[0.04359209, 0.1771886, 0.091143146, 0.062854...","[0.2153729796409607, 0.1547222137451172, 0.387...","[9.725599, 10.581477, 13.880341]",1,0


In [79]:
predicted1["cosine_sim"][0]

[0.42473626136779785,
 0.364050030708313,
 0.3477550148963928,
 0.3942415714263916,
 0.3710247874259949,
 0.1856902837753296,
 0.3519207239151001]

In [82]:
predicted1["euclidean_dis"][0]

[14.56386, 15.262213, 17.398178, 14.272491, 13.339654, 9.336262, 15.720998]

In [83]:
def accuracy(target, predicted):
    acc = (target==predicted).sum()/len(target)
    return acc

In [88]:
acc_ecu1 = accuracy(predicted1["target"], predicted1["pred_idx_euc"])
acc_ecu2 = accuracy(predicted2["target"], predicted2["pred_idx_euc"])
acc_ecu3 = accuracy(predicted3["target"], predicted3["pred_idx_euc"])
acc_ecu4 = accuracy(predicted4["target"], predicted4["pred_idx_euc"])
acc_ecu5 = accuracy(predicted5["target"], predicted5["pred_idx_euc"])
acc_ecu6 = accuracy(predicted6["target"], predicted6["pred_idx_euc"])
print((acc_ecu1+acc_ecu2+acc_ecu2+acc_ecu3+acc_ecu4+acc_ecu5+acc_ecu6)/6)

0.5206560318594374


In [110]:
acc_cos1 = accuracy(predicted1["target"], predicted1["pred_idx_cos"])
acc_cos2 = accuracy(predicted2["target"], predicted2["pred_idx_cos"])
acc_cos3 = accuracy(predicted3["target"], predicted3["pred_idx_cos"])
acc_cos4 = accuracy(predicted4["target"], predicted4["pred_idx_cos"])
acc_cos5 = accuracy(predicted5["target"], predicted5["pred_idx_cos"])
acc_cos6 = accuracy(predicted6["target"], predicted6["pred_idx_cos"])
print(acc_cos1,acc_cos2,acc_cos3,acc_cos4,acc_cos5,acc_cos6)
print((acc_cos1+acc_cos2+acc_cos3+acc_cos4+acc_cos5+acc_cos6)/6)

0.5781333333333334 0.6113333333333333 0.5636 0.5370666666666667 0.5283333333333333 0.5520361990950227
0.5617504776269482


In [95]:
predicted1.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent1.csv", index=None)
predicted2.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent2.csv", index=None)
predicted3.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent3.csv", index=None)
predicted4.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent4.csv", index=None)
predicted5.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent5.csv", index=None)
predicted6.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent6.csv", index=None)

In [91]:
frames = [predicted1, predicted2, predicted3, predicted4,predicted5,predicted6]

In [93]:
predicted = pd.concat(frames)

In [96]:
predicted.to_csv("/content/drive/MyDrive/data/temp/train_detect_sent.csv", index=None)

In [ ]:
del predicted1, predicted2, predicted3, predicted4,predicted5,predicted6

In [101]:
predicted.tail()

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
87593,734,The main international airport serving Kathman...,Who operates flights between Kathmandu and Ist...,Turkish Airlines,[The main international airport serving Kathma...,4,"[[0.10149852, 0.18207367, 0.1680953, 0.1682793...","[[0.13340311, 0.08044777, 0.12206173, 0.149480...","[0.27631676197052, 0.3307735323905945, 0.26635...","[15.749717, 11.977074, 14.941778, 15.306896, 7...",4,4
87594,229,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon,"[Kathmandu Metropolitan City (KMC), in order t...",1,"[[0.061106615, 0.11834025, 0.080840126, -0.010...","[[0.14494264, 0.14140247, 0.015738873, 0.08947...","[0.27094709873199463, 0.23466825485229492, 0.2...","[11.593131, 9.120165, 15.337, 14.724384]",1,1
87595,414,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,"[Kathmandu Metropolitan City (KMC), in order t...",2,"[[0.061106615, 0.11834025, 0.080840126, -0.010...","[[0.057453785, 0.07620494, 0.09973739, 0.00248...","[0.4904106855392456, 0.49210208654403687, 0.48...","[18.325274, 16.419464, 23.756025, 24.184345]",2,1
87596,476,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,"[Kathmandu Metropolitan City (KMC), in order t...",2,"[[0.061106615, 0.11834025, 0.080840126, -0.010...","[[0.0667483, 0.023414522, 0.036765143, 0.02317...","[0.32654869556427, 0.33208364248275757, 0.3030...","[13.438961, 12.213377, 16.69118, 15.916579]",2,1
87597,199,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,"[Kathmandu Metropolitan City (KMC), in order t...",1,"[[0.061106615, 0.11834025, 0.080840126, -0.010...","[[0.042654388, 0.059282392, 0.04575804, 0.0977...","[0.28552085161209106, 0.2907397747039795, 0.28...","[12.050905, 11.00305, 15.967188, 14.104914]",3,1


In [141]:
predicted.iloc[75206,:]

answer_start                                                   384
context          One example of omnidirectional antennas is the...
question         In which direction would you expect to find a ...
text                                                  horizontally
sentences        [One example of omnidirectional antennas is th...
target                                                           2
sent_emb         [[0.04644427, 0.16050649, 0.16383033, 0.044872...
quest_emb        [[0.068846576, 0.024703005, 0.061177757, -0.05...
cosine_sim       [0.37727469205856323, 0.3294129967689514, 0.34...
euclidean_dis                      [16.36916, 13.730917, 17.12673]
pred_idx_cos                                                     1
pred_idx_euc                                                     1
Name: 75206, dtype: object

In [104]:
ct,k = 0,0
for i in range(predicted.shape[0]):
    if predicted.iloc[i,10] != predicted.iloc[i,5]:
        k += 1
        if predicted.iloc[i,11] == predicted.iloc[i,5]:
            ct += 1

In [105]:
ct, k

(9732, 38366)

In [111]:
print(predicted.shape[0])

87597


In [106]:
label = []
for i in range(predicted.shape[0]):
    if predicted.iloc[i,10] == predicted.iloc[i,11]:
        label.append(predicted.iloc[i,10])
    else:
        label.append((predicted.iloc[i,10],predicted.iloc[i,10]))

In [113]:
ct = 0
# for i in range(predicted.shape[0]):
for i in range(75206):
    item = predicted["target"][i]
    try:
        if label[i] == predicted["target"][i]: ct +=1
    except:
        if item in label[i]: ct +=1

In [114]:
ct, ct/75206

(42409, 0.5639044757067255)

In [152]:
predicted = pd.read_csv("/content/drive/MyDrive/data/temp/train_detect_sent.csv").reset_index(drop=True)

In [157]:
predicted.shape

(87597, 12)

In [158]:
doc = en_nlp(predicted.iloc[0,1])

In [159]:
predicted.iloc[0,1]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [160]:
predicted.iloc[0,2]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [161]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [162]:
[to_nltk_tree(sent.root).pretty_print()  for sent in en_nlp(predicted.iloc[0,2]).sents]

                  appear                             
  __________________|____________________________     
 |      |      |    |         |           |      in  
 |      |      |    |         |           |      |    
 |      |      |    To       Mary         in   France
 |      |      |    |      ___|_____      |      |    
did allegedly  ?   whom  the      Virgin 1858 Lourdes



[None]

In [163]:
[to_nltk_tree(sent.root) .pretty_print() for sent in doc.sents][5]

                    has                              
        _____________|_________________               
       |         |   |  school     character         
       |         |   |    |      ______|________      
Architecturally  ,   .   the    a            Catholic

                          is                                  
  ________________________|_______________                     
 |    |            |                    statue                
 |    |            |                  ____|_________           
 |    |           dome               |    |         of        
 |    |    ________|______           |    |         |          
 |    |   |            Building      |    |        Mary       
 |    |   |     __________|______    |    |      ___|_____     
Atop  .  gold the        Main    's  a  golden the      Virgin

                                                is                                                  
  ______________________________________________|_____

In [164]:
for sent in doc.sents:
    roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]
    print(roots)

['has', 'has']
['is', 'is', 'of']
['in', 'of', 'fac', 'is', 'of', 'with', 'with', 'legend']
['to', 'is', 'of']
['behind', 'is', 'grotto', 'of', 'pray']
['is', 'is', 'of', 'at', 'lourd', 'appear', 'to']
['at', 'of', 'in', 'through', 'statu', 'is', 'of']


In [165]:
def match_roots(x):
    question = x["question"].lower()
    sentences = en_nlp(x["context"].lower()).sents
    
    question_root = st.stem(str([sent.root for sent in en_nlp(question).sents][0]))
    
    li = []
    for i,sent in enumerate(sentences):
        roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]

        if question_root in roots: 
            for k,item in enumerate(ast.literal_eval(x["sentences"])):
                if str(sent) in item.lower(): 
                    li.append(k)
    return li

In [166]:
predicted["question"][100]

'In what year did the team lead by Knute Rockne win the Rose Bowl?'

In [167]:
predicted["question"][21493]

'The end of what road was once home to Newgate Prison?'

In [168]:
predicted["context"][21493]

"10th Street (40°44′03″N 74°00′11″W\ufeff / \ufeff40.7342580°N 74.0029670°W\ufeff / 40.7342580; -74.0029670) begins at the FDR Drive and Avenue C. West of Sixth Avenue, it turns southward about 40 degrees to join the Greenwich Village street grid and continue to West Street on the Hudson River. Because West 4th Street turns northward at Sixth Avenue, it intersects 10th, 11th and 12th and 13th Streets in the West Village. The M8 bus operates on 10th Street in both directions between Avenue D and Avenue A, and eastbound between West Street and Sixth Avenue. 10th Street has an eastbound bike lane from West Street to the East River. In 2009, the two-way section of 10th Street between Avenue A and the East River had bicycle markings and sharrows installed, but it still has no dedicated bike lane. West 10th Street was previously named Amos Street for Richard Amos. The end of West 10th Street toward the Hudson River was once the home of Newgate Prison, New York City's first prison and the Uni

In [169]:
predicted["root_match_idx"] = predicted.apply(match_roots, axis = 1)

In [170]:
predicted["root_match_idx_first"]= predicted["root_match_idx"].apply(lambda x: x[0] if len(x)>0 else 0)

In [171]:
(predicted["root_match_idx_first"]==predicted["target"]).sum()/predicted.shape[0]

0.40405493338812976

In [148]:
question = predicted["question"][21493].lower()
sentences = en_nlp(predicted["context"][21493].lower()).sents
    
question_root = st.stem(str([sent.root for sent in en_nlp(question).sents][0]))
    
li = []
for i,sent in enumerate(sentences):
    roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]
    print(roots)

    if question_root in roots: li.append(i)

['street']
['°']
['w\ufeff']
['at', 'driv', 'of', 'turn', 'turn', 'join', 'to', 'on']
['turn', 'at', 'intersect', 'intersect', 'in']
['op', 'on', 'in', 'between', 'op', 'between', 'street']
['has', 'has', 'from', 'to']
['had', 'of', 'between', 'a', 'had', 'mark', 'has', 'has']
['nam', 'for']
['was', 'of', 'toward', 'was', 'of', 'prison', 'prison']


In [172]:
ast.literal_eval(predicted["sentences"][21493])

['10th Street (40°44′03″N 74°00′11″W\ufeff / \ufeff40.7342580°N 74.0029670°W\ufeff / 40.7342580; -74.0029670) begins at the FDR Drive and Avenue C. West of Sixth Avenue, it turns southward about 40 degrees to join the Greenwich Village street grid and continue to West Street on the Hudson River.',
 'Because West 4th Street turns northward at Sixth Avenue, it intersects 10th, 11th and 12th and 13th Streets in the West Village.',
 'The M8 bus operates on 10th Street in both directions between Avenue D and Avenue A, and eastbound between West Street and Sixth Avenue.',
 '10th Street has an eastbound bike lane from West Street to the East River.',
 'In 2009, the two-way section of 10th Street between Avenue A and the East River had bicycle markings and sharrows installed, but it still has no dedicated bike lane.',
 'West 10th Street was previously named Amos Street for Richard Amos.',
 "The end of West 10th Street toward the Hudson River was once the home of Newgate Prison, New York City's

In [150]:
predicted["context"][21493]

"10th Street (40°44′03″N 74°00′11″W\ufeff / \ufeff40.7342580°N 74.0029670°W\ufeff / 40.7342580; -74.0029670) begins at the FDR Drive and Avenue C. West of Sixth Avenue, it turns southward about 40 degrees to join the Greenwich Village street grid and continue to West Street on the Hudson River. Because West 4th Street turns northward at Sixth Avenue, it intersects 10th, 11th and 12th and 13th Streets in the West Village. The M8 bus operates on 10th Street in both directions between Avenue D and Avenue A, and eastbound between West Street and Sixth Avenue. 10th Street has an eastbound bike lane from West Street to the East River. In 2009, the two-way section of 10th Street between Avenue A and the East River had bicycle markings and sharrows installed, but it still has no dedicated bike lane. West 10th Street was previously named Amos Street for Richard Amos. The end of West 10th Street toward the Hudson River was once the home of Newgate Prison, New York City's first prison and the Uni

In [173]:
en_nlp = spacy.load('en')
sentences = en_nlp(predicted["context"][21493].lower()).sents

In [174]:
for item in sentences:
    print(item)

10th street (40°44′03″n 74°
00′11″w﻿ / ﻿40.7342580°n 74.0029670°
w﻿ / 40.7342580
; -74.0029670) begins at the fdr drive and avenue c. west of sixth avenue, it turns southward about 40 degrees to join the greenwich village street grid and continue to west street on the hudson river.
because west 4th street turns northward at sixth avenue, it intersects 10th, 11th and 12th and 13th streets in the west village.
the m8 bus operates on 10th street in both directions between avenue d and avenue a, and eastbound between west street and sixth avenue.
10th street has an eastbound bike lane from west street to the east river.
in 2009, the two-way section of 10th street between avenue a and the east river had bicycle markings and sharrows installed, but it still has no dedicated bike lane.
west 10th street was previously named amos street for richard amos.
the end of west 10th street toward the hudson river was once the home of newgate prison, new york city's first prison and the united states' s

In [175]:
TfidfVectorizer(predicted["sentences"][0], ngram_range=(1,2))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input="['Architecturally, the school has a Catholic "
                      'character.\', "Atop the Main Building\'s gold dome is a '
                      'golden statue of the Virgin Mary.", \'Immediately in '
                      'front of the Main Building and facing it, is a copper '
                      'statue of Christ with arms upraised with the legend '
                      '"Ve...
                      'the end of the main drive (and in a direct line that '
                      'connects through 3 statues and the Gold Dome), is a '
                      "simple, modern stone statue of Mary.']",
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublin